In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

In [0]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 50 # 訓練的 epochs 數量
LEARNING_RATE = 1e-3
optimizers = [RMSprop, Adam]

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [12]:
x_train.shape[1:]

(32, 32, 3)

In [0]:
def build_cnn(input_shape, output_units=10, num_neurons=512):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(num_neurons))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(output_units))
  model.add(Activation('softmax'))

  return model

In [16]:
results = {}

for _batch in batch_size:
  keras.backend.clear_session()
  print("Experiment with batch size = {}".format(_batch))
  model = build_cnn(input_shape=x_train.shape[1:])
  model.summary()
  
  optimizer = keras.optimizers.RMSprop(lr=LEARNING_RATE)

  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  model.fit(x_train, y_train,                      
            batch_size=_batch,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))
  
  # Collect results
  train_loss = model.history.history["loss"]
  valid_loss = model.history.history["val_loss"]
  train_acc = model.history.history["acc"]
  valid_acc = model.history.history["val_acc"]
  
  exp_name_tag = "batch size:{}".format(_batch)  
  score = model.evaluate(x_test, y_test, verbose=0)
  results[exp_name_tag] = {'train-loss': train_loss,
                           'valid-loss': valid_loss,
                           'train-acc': train_acc,
                           'valid-acc': valid_acc,
                           'score': score,}
  print(exp_name_tag)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

Experiment with batch size = 32
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)     

In [19]:
results = {}

for _epoch in epochs:
  keras.backend.clear_session()
  print("Experiment with _epoch = {}".format(_epoch))
  model = build_cnn(input_shape=x_train.shape[1:])
  model.summary()
  print("----------------------------------------")
  
  optimizer = keras.optimizers.RMSprop(lr=LEARNING_RATE)

  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  model.fit(x_train, y_train,                      
            batch_size=_batch,
            epochs=_epoch,
            verbose=1,
            validation_data=(x_test, y_test))
  
  # Collect results
  train_loss = model.history.history["loss"]
  valid_loss = model.history.history["val_loss"]
  train_acc = model.history.history["acc"]
  valid_acc = model.history.history["val_acc"]
  
  exp_name_tag = "epoch:{}".format(_epoch)  
  score = model.evaluate(x_test, y_test, verbose=0)
  results[exp_name_tag] = {'train-loss': train_loss,
                           'valid-loss': valid_loss,
                           'train-acc': train_acc,
                           'valid-acc': valid_acc,
                           'score': score,}
  print(exp_name_tag)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

Experiment with _epoch = 10
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)         

In [23]:
results = {}

for _optimizer in optimizers:
  keras.backend.clear_session()
  print("Experiment with _optimizer = {}".format(_optimizer.__name__))
  model = build_cnn(input_shape=x_train.shape[1:])
  model.summary()
  print("----------------------------------------")
  
  optimizer = _optimizer(lr=LEARNING_RATE)

  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  model.fit(x_train, y_train,                      
            batch_size=_batch,
            epochs=_epoch,
            verbose=1,
            validation_data=(x_test, y_test))
  
  # Collect results
  train_loss = model.history.history["loss"]
  valid_loss = model.history.history["val_loss"]
  train_acc = model.history.history["acc"]
  valid_acc = model.history.history["val_acc"]
  
  exp_name_tag = "_optimizer:{}".format(_optimizer)  
  score = model.evaluate(x_test, y_test, verbose=0)
  results[exp_name_tag] = {'train-loss': train_loss,
                           'valid-loss': valid_loss,
                           'train-acc': train_acc,
                           'valid-acc': valid_acc,
                           'score': score,}
  print(exp_name_tag)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

Experiment with _optimizer = RMSprop
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

In [0]:
"""
batch size 越大, 表現越好。
epoch 越多, 表現越好
optimizer 使用 adam比rmsprop的表現好
